In [1]:
import json
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np

import re
import sys
import itertools
from torch.utils.data import Dataset, DataLoader

import random
import os
import pickle
import codecs
from gensim import corpora
import gensim

In [6]:
with open('data/processed_trumptwitterarchive.txt', 'r') as data_file:
    json_data = data_file.read()
data = json.loads(json_data)
len(data)

42692

In [15]:
sent = pd.read_csv('data/sentimentAnalysis/senti.csv')
df = pd.DataFrame()

In [21]:
df['time'] = sent['time'].apply(lambda x: x[:-6])
df['texts'] = sent['texts']
df['retweet_count'] = sent['retweet_count']
df['favorite_count'] = sent['favorite_count']

In [22]:
df.head()

,time,texts,retweet_count,favorite_count
0,2019-11-16 15:18:27,dow hit first time ever highest ever gee pelos...,4301,15011
1,2019-11-16 13:56:23,rt armyst lt clint lorance one two u army offi...,9149,0
2,2019-11-16 13:51:34,louisiana vote today great governor,7311,24555
3,2019-11-16 13:08:14,good morning louisiana poll open atam get vote...,10699,30751
4,2019-11-16 06:03:27,rt big day louisiana tomorrow get vote replace...,10084,0


In [25]:
df['begin_time'] = df['time'].apply(lambda x: x[:-2]+'00')
df['begin_time'] = df['begin_time'].apply(lambda x: pd.to_datetime(x))
df['end_time'] = df['begin_time'] + timedelta(minutes=5)

In [26]:
df.head()

,time,texts,retweet_count,favorite_count,begin_time,end_time
0,2019-11-16 15:18:27,dow hit first time ever highest ever gee pelos...,4301,15011,2019-11-16 15:18:00,2019-11-16 15:23:00
1,2019-11-16 13:56:23,rt armyst lt clint lorance one two u army offi...,9149,0,2019-11-16 13:56:00,2019-11-16 14:01:00
2,2019-11-16 13:51:34,louisiana vote today great governor,7311,24555,2019-11-16 13:51:00,2019-11-16 13:56:00
3,2019-11-16 13:08:14,good morning louisiana poll open atam get vote...,10699,30751,2019-11-16 13:08:00,2019-11-16 13:13:00
4,2019-11-16 06:03:27,rt big day louisiana tomorrow get vote replace...,10084,0,2019-11-16 06:03:00,2019-11-16 06:08:00


In [24]:
data_econ = pd.read_csv('data/all_cleaned_SPX_data.csv')
data_econ['datetime'] = pd.to_datetime(data_econ['datetime'])
df_econ = data_econ[['datetime', 'close']]

In [27]:
df_merge1 = pd.merge(df, df_econ, left_on = 'begin_time', right_on='datetime').drop(['datetime'], axis=1)
df_merge2 = pd.merge(df_merge1, df_econ, left_on = 'end_time', right_on='datetime').drop(['datetime'], axis=1)
df_merge2.head()

,time,texts,retweet_count,favorite_count,begin_time,end_time,close_x,close_y
0,2019-11-07 15:43:29,stock market big today new record enjoy,20211,114187,2019-11-07 15:43:00,2019-11-07 15:48:00,3080.94,3082.54
1,2019-11-07 15:41:53,radical left dems lamestream medium trying mak...,18328,72619,2019-11-07 15:41:00,2019-11-07 15:46:00,3082.59,3081.32
2,2019-11-07 15:27:57,amazon washington post three lowlife reporter ...,24007,83333,2019-11-07 15:27:00,2019-11-07 15:32:00,3082.41,3083.63
3,2019-11-07 15:18:53,thank #MAGA,14962,65540,2019-11-07 15:18:00,2019-11-07 15:23:00,3082.46,3082.57
4,2019-11-07 15:16:15,explained next week fake hearing trial house i...,35482,132078,2019-11-07 15:16:00,2019-11-07 15:21:00,3084.20,3080.28


In [29]:
len(df_merge2)

9312

In [30]:
df_merge2['label'] = (df_merge2['close_y'] > df_merge2['close_x'])*1

In [31]:
df_merge2.t

,time,texts,retweet_count,favorite_count,begin_time,end_time,close_x,close_y,label
0,2019-11-07 15:43:29,stock market big today new record enjoy,20211,114187,2019-11-07 15:43:00,2019-11-07 15:48:00,3080.94,3082.54,1
1,2019-11-07 15:41:53,radical left dems lamestream medium trying mak...,18328,72619,2019-11-07 15:41:00,2019-11-07 15:46:00,3082.59,3081.32,0
2,2019-11-07 15:27:57,amazon washington post three lowlife reporter ...,24007,83333,2019-11-07 15:27:00,2019-11-07 15:32:00,3082.41,3083.63,1
3,2019-11-07 15:18:53,thank #MAGA,14962,65540,2019-11-07 15:18:00,2019-11-07 15:23:00,3082.46,3082.57,1
4,2019-11-07 15:16:15,explained next week fake hearing trial house i...,35482,132078,2019-11-07 15:16:00,2019-11-07 15:21:00,3084.20,3080.28,0
